In [3]:
# use this file to build the dataframes 
# then compile all of the code into a single block for each question in a separate file

# Questions to answer.
# 1) Wide Receivers with at least 5 targets per game for each week with defined year range 
# 2) Wide Receivers with at least 30 recptions for the year ranked by yards per recption

In [4]:
import nfl_data_py as nfl
import pandas as pd

# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)        # Set the display width to a large number

import numpy as np
from IPython.display import HTML

In [5]:
from IPython.display import display, HTML

# Use Pandas Styler to create a scrollable DataFrame with sticky headers
def enable_styler_scrolling_with_sticky_header(df):
    
    styled_df = df.style.set_table_styles(
        [
            {'selector': 'thead th', 'props': [('position', 'sticky'),
                                               ('top', '0'),
                                               ('background-color', 'white'),
                                               ('z-index', '1'),
                                               ('border', '1px solid black'),
                                               ('text-align', 'left')]},
            {'selector': 'tbody td', 'props': [('border', '1px solid black'),
                                               ('text-align', 'left'),
                                               ('padding', '4px'),
                                               ('font-family', 'Arial, sans-serif'),
                                               ('font-size', '12px')]},
            {'selector': 'table', 'props': [('border-collapse', 'collapse'),
                                            ('width', '100%'),
                                            ('table-layout', 'fixed')]}
        ]
    ).set_properties(
        **{
            'white-space': 'nowrap',
            'overflow': 'hidden'
        }
    )

    # Use to_html() to render the Styler object to HTML
    return display(HTML(f'<div style="overflow-x: auto;">{styled_df.to_html()}</div>'))

In [6]:
# see available columns for weekly data

weeks = nfl.see_weekly_cols()
# print(len(weeks)) 

# view the available categories
# for week in weeks:
#     print(week)

In [7]:
# Base columns for all positions
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name', 
    'position', 'position_group', 'recent_team', 'opponent_team',
    'fantasy_points', 'fantasy_points_ppr'
]
# print(base_columns)

In [8]:
# WR-specific columns (receiving-related)
wr_columns = [
    'receptions', 'targets', 'receiving_yards', 'receiving_tds', 
    'receiving_fumbles', 'receiving_fumbles_lost', 
    'receiving_air_yards', 'receiving_yards_after_catch', 
    'receiving_first_downs', 'receiving_epa', 
    'receiving_2pt_conversions', 'racr', 'target_share', 
    'air_yards_share', 'wopr'
]

# Define the WR-specific target-related columns
wr_target_columns = ['targets','target_share','receptions','receiving_yards']

# Combine base columns with WR-specific column
wr_all_columns = base_columns + wr_columns
wr_tgt_columns = base_columns + wr_target_columns

In [9]:
# Import all relevant data for all positions
nfl_data = nfl.import_weekly_data(
    years=[2023], 
    columns=base_columns + wr_columns
)

Downcasting floats.


In [10]:
# Filter to show only WR positions
wr_data = nfl_data[nfl_data['position'] == 'WR']

# Select only the relevant columns for WRs
wr_tgt_data = wr_data[wr_tgt_columns]

# Sort by week for easier viewing
wr_tgt_data = wr_tgt_data.sort_values(by='week')

In [11]:
# Filter WRs with at least 5 targets per game
wr_with_min_5_targets = wr_tgt_data[wr_tgt_data['targets'] >= 5]

# Sort by week for better analysis
wr_with_min_5_targets_sorted = wr_with_min_5_targets.sort_values(by=['week', 'targets'], ascending=[True, False])

# Display the filtered data
# display(HTML(wr_with_min_5_targets_sorted.to_html(index=False)))

In [12]:
# Remove the 'player_id' and 'position_group' columns
wr_filtered_data = wr_with_min_5_targets_sorted.drop(columns=['player_id', 'position_group'])

# Format decimals to two decimal places for all columns
wr_filtered_data = wr_filtered_data.round(2)

# Specifically format 'fantasy_points' and 'fantasy_points_ppr' as strings with two decimal places
wr_filtered_data['fantasy_points'] = wr_filtered_data['fantasy_points'].apply(lambda x: f"{x:.2f}")
wr_filtered_data['fantasy_points_ppr'] = wr_filtered_data['fantasy_points_ppr'].apply(lambda x: f"{x:.2f}")


# ** function ** 
# Display the refined data
display(HTML(wr_filtered_data.to_html(index=False)))

season,season_type,week,player_name,position,recent_team,opponent_team,fantasy_points,fantasy_points_ppr,targets,target_share,receptions,receiving_yards
2023,REG,1,T.Hill,WR,MIA,LAC,33.50,44.50,15,0.34,11,215.0
2023,REG,1,P.Nacua,WR,LA,SEA,11.90,21.90,15,0.41,10,119.0
2023,REG,1,S.Diggs,WR,BUF,NYJ,16.20,26.20,13,0.33,10,102.0
2023,REG,1,D.Hopkins,WR,TEN,NO,6.50,13.50,13,0.39,7,65.0
2023,REG,1,J.Jefferson,WR,MIN,TB,15.00,24.00,12,0.27,9,150.0
2023,REG,1,M.Pittman,WR,IND,JAX,15.70,23.70,11,0.28,8,97.0
2023,REG,1,K.Bourne,WR,NE,PHI,18.40,24.40,11,0.20,6,64.0
2023,REG,1,C.Ridley,WR,JAX,IND,16.10,24.10,11,0.34,8,101.0
2023,REG,1,N.Collins,WR,HOU,BAL,8.00,14.00,11,0.26,6,80.0
2023,REG,1,C.Olave,WR,NO,TEN,11.20,19.20,10,0.30,8,112.0


In [13]:
# 2) Wide Receivers with at least 30 recptions for the year ranked by yards per recption

In [35]:

# Define the years of interest
years = [2020, 2021, 2022, 2023]

# Load data for the specified years
nfl_data_selected_years = nfl.import_weekly_data(
    years=years,
    columns=wr_all_columns
)

# Filter to show only WR positions
wr_data_selected_years = nfl_data_selected_years[nfl_data_selected_years['position'] == 'WR']

# Group by season and player, then sum the relevant statistics
wr_grouped_data = wr_data_selected_years.groupby(['season', 'player_name', 'recent_team'], as_index=False).agg({
    'receptions': 'sum',
    'receiving_yards': 'sum'
})

# Filter WRs with at least 30 receptions in a season and create a copy of the DataFrame to avoid the warning
wr_filtered_by_receptions = wr_grouped_data[wr_grouped_data['receptions'] >= 30].copy()

# Calculate yards per reception using .loc[] to avoid the SettingWithCopyWarning
wr_filtered_by_receptions['yards_per_reception'] = wr_filtered_by_receptions['receiving_yards'] / wr_filtered_by_receptions['receptions']

# Rank WRs by yards per reception in descending order
wr_ranked_by_yards_per_reception = wr_filtered_by_receptions.sort_values(by=['season', 'yards_per_reception'], ascending=[False, False])

# Format decimals to two decimal places
wr_ranked_by_yards_per_reception = wr_ranked_by_yards_per_reception.round(2)

# Display the ranked WRs
display(HTML(wr_ranked_by_yards_per_reception.to_html(index=False)))


Downcasting floats.


season,player_name,recent_team,receptions,receiving_yards,yards_per_reception
2023,B.Aiyuk,SF,84,1491.0,17.75
2023,G.Pickens,PIT,68,1190.0,17.50
2023,A.Cooper,CLE,76,1309.0,17.22
2023,N.Brown,HOU,33,567.0,17.18
2023,D.Metcalf,SEA,66,1114.0,16.88
2023,J.Watson,KC,33,550.0,16.67
2023,G.Davis,BUF,45,746.0,16.58
2023,M.Evans,TB,90,1450.0,16.11
2023,A.Pierce,IND,32,514.0,16.06
2023,N.Collins,HOU,91,1461.0,16.05
